The field of genomics has seen massive growth in the past few years because of advancements in sequencing technologies that have pushed genomic data in the wave of big data. ML techniques, with their ability to analyze large-scale data, can turn this complex genomic data into biological insights that can in turn convert into useful products. ML algorithms have been widely used in biology and genomics to apply to complex multi-dimensional datasets for the building of predictive models to solve complex biological problems such as disorder risk prediction, mental illness prediction, diagnosis, treatment, and so on. This chapter introduces the main ML algorithms and libraries that are commonly used in genomic data analysis. By the end of the chapter, you will know what supervised and unsupervised ML methods are, understand the most common ML algorithms and libraries for genomics, and know when and how to use them. You will know how to build a predictive model using genomics data and be aware of the challenges encountered by ML algorithms for genomics and potential solutions for addressing the same.